In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import matplotlib.pyplot as plt
import random
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# 定义数据增强方法：CutMix
def cutmix_data(images, labels, alpha):
    # 随机选择一个样本
    indices = torch.randperm(images.size(0))
    shuffled_images = images[indices]
    shuffled_labels = labels[indices]

    # 计算剪切区域的大小
    lam = np.random.beta(alpha, alpha)
    cut_w = int(images.size(2) * np.sqrt(1 - lam))
    cut_h = int(images.size(3) * np.sqrt(1 - lam))

    # 随机选择剪切区域的位置
    cx = np.random.randint(images.size(2))
    cy = np.random.randint(images.size(3))
    x1 = np.clip(cx - cut_w // 2, 0, images.size(2))
    x2 = np.clip(cx + cut_w // 2, 0, images.size(2))
    y1 = np.clip(cy - cut_h // 2, 0, images.size(3))
    y2 = np.clip(cy + cut_h // 2, 0, images.size(3))

    # 剪切区域替换为随机样本的剪切区域
    images[:, :, x1:x2, y1:y2] = shuffled_images[:, :, x1:x2, y1:y2]

    # 计算新的标签
    lam = 1 - ((x2 - x1) * (y2 - y1) / (images.size(2) * images.size(3)))
    labels = (1 - lam) * labels + lam * shuffled_labels

    return images, labels

In [ ]:
# 定义数据增强方法：Cutout
def cutout_data(images, labels, n_holes, length):
    h = images.size(2)
    w = images.size(3)

    for _ in range(n_holes):
        y = np.random.randint(h)
        x = np.random.randint(w)

        y1 = np.clip(y - length // 2, 0, h)
        y2 = np.clip(y + length // 2, 0, h)
        x1 = np.clip(x - length // 2, 0, w)
        x2 = np.clip(x + length // 2, 0, w)

        images[:, :, y1:y2, x1:x2] = 0

    return images, labels

In [ ]:
def mixup_data(images, labels, alpha):
    # 随机选择另一个样本
    indices = torch.randperm(images.size(0))
    shuffled_images = images[indices]
    shuffled_labels = labels[indices]

    # 计算混合比例
    lam = np.random.beta(alpha, alpha)
    lam = max(lam, 1 - lam)

    # 执行mixup
    images = lam * images + (1 - lam) * shuffled_images
    labels = lam * labels + (1 - lam) * shuffled_labels

    return images, labels

In [ ]:
# 设置随机种子以便结果可重现
torch.manual_seed(42)
np.random.seed(42)
# 设置训练和测试的批处理大小
batch_size = 64


In [ ]:
# 加载CIFAR-100数据集
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

In [ ]:
# 定义ResNet-18模型
model = resnet18(pretrained=False, num_classes=100)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

# 创建 SummaryWriter 对象
writer = SummaryWriter()
# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 100

train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        # 写入训练 loss 值到 Tensorboard
        writer.add_scalar('Train/Loss', train_loss/(batch_idx+1), epoch)
        # 写入训练准确率到 Tensorboard
        train_accuracy = 100. * correct / total
        writer.add_scalar('Train/Accuracy', train_accuracy, epoch)
        if (batch_idx + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch+1, num_epochs, batch_idx+1, len(trainloader),
                          train_loss/(batch_idx+1), 100. * correct / total))
    
    # 在每个epoch结束后记录训练和测试的损失和准确率
    train_loss /= len(trainloader)
    train_accuracy = 100. * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    # 保存模型
torch.save(model, './baseline_ResNet-18')

# 测试模型
model.eval()
test_loss = 0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        test_total += targets.size(0)
        test_correct += predicted.eq(targets).sum().item()

    # 在每个epoch结束后记录测试的损失和准确率
    test_loss /= len(testloader)
    test_accuracy = 100. * test_correct / test_total
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

# 绘制训练和测试损失变化图
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

# 绘制训练和测试准确率变化图
plt.figure()
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR100
from torchvision.transforms import ToTensor, Normalize, RandomCrop, RandomHorizontalFlip
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# 设置随机种子
torch.manual_seed(42)

# 定义超参数
batch_size = 64
learning_rate = 0.1
num_epochs = 100

# 定义设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载CIFAR-100数据集并进行预处理
transform_train = transforms.Compose([
    RandomCrop(32, padding=4),
    RandomHorizontalFlip(),
    transforms.ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR100(root='./data', train=True, transform=transform_train, download=True)
test_dataset = CIFAR100(root='./data', train=False, transform=transform_test)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import math

# 定义超参数
patch_size = 16

# 定义Vision Transformer模型
class VisionTransformer(nn.Module):
    def __init__(self, d_model=512, nhead=8, num_layers=1, num_classes=100):
        super(VisionTransformer, self).__init__()

        self.patch_size = patch_size
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers

        self.patch_embedding = nn.Linear(patch_size * patch_size * 3, d_model)
        self.positional_embedding = nn.Parameter(torch.randn(1, 32 * 32 // patch_size // patch_size + 1, d_model))
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, 3, 32 // self.patch_size, self.patch_size, 32 // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 3, 5, 1).contiguous()
        x = x.view(batch_size, -1, self.patch_size * self.patch_size * 3)
        x = self.patch_embedding(x)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.positional_embedding
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.mean(dim=0)
        x = self.fc(x)
        return x

# 创建模型实例
model = VisionTransformer(d_model=512, nhead=8, num_layers=3, num_classes=100)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# 记录训练过程的指标
train_loss_history = []
test_loss_history = []
train_acc_history = []
test_acc_history = []

# 使用TensorBoard可视化训练过程
writer = SummaryWriter(log_dir='./logs')

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = 100.0 * correct / total

    train_loss_history.append(avg_train_loss)
    train_acc_history.append(avg_train_acc)

    writer.add_scalar('Train Loss', avg_train_loss, epoch + 1)
    writer.add_scalar('Train Accuracy', avg_train_acc, epoch + 1)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_acc:.2f}%')

    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    avg_test_acc = 100.0 * test_correct / test_total

    test_loss_history.append(avg_test_loss)
    test_acc_history.append(avg_test_acc)

    writer.add_scalar('Test Loss', avg_test_loss, epoch + 1)
    writer.add_scalar('Test Accuracy', avg_test_acc, epoch + 1)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Test Loss: {avg_test_loss:.4f}, Test Accuracy: {avg_test_acc:.2f}%')

    scheduler.step()

# 保存模型
torch.save(model.state_dict(), 'transformer_cifar100.pth')

# 关闭TensorBoard写入器
writer.close()

# 可视化训练过程
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Train Loss')
plt.plot(test_loss_history, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_acc_history, label='Train Accuracy')
plt.plot(test_acc_history, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# 可视化样本图像
def imshow(img):
    img = img / 2 + 0.5     # 反归一化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

In [ ]:
# # 获取样本数据
# 加载三张训练样本
sample_loader = torch.utils.data.DataLoader(trainset, batch_size=3, shuffle=True, num_workers=2)
example_data, example_targets = next(iter(sample_loader))
# example_data = example_data.to(device)
# # 获取样本和标签
# samples, labels = next(iter(sample_loader))
# # samples = samples.to(device)
# examples = enumerate(trainloader)
# batch_idx, (example_data, example_targets) = next(examples)

# 将样本数据进行cutmix处理
cutmix_images, cutmix_labels = cutmix_data(example_data.clone(), example_targets.clone(), alpha=1.0)

# 将样本数据进行cutout处理
cutout_images, cutout_labels = cutout_data(example_data.clone(), example_targets.clone(), n_holes=1, length=16)

# 将样本数据进行mixup处理
mixup_images, mixup_labels = mixup_data(example_data.clone(), example_targets.clone(), alpha=1.0)

In [ ]:
print("Original Images:")
imshow(torchvision.utils.make_grid(example_data))
print("CutMix Images:")
imshow(torchvision.utils.make_grid(cutmix_images))
print("Cutout Images:")
imshow(torchvision.utils.make_grid(cutout_images))
print("Mixup Images:")
imshow(torchvision.utils.make_grid(mixup_images))